In [14]:
import gurobipy as gb
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import function_library_assignment_1 as fnc

import warnings
warnings.simplefilter(action='ignore', category=FutureWarning) #filtering FutureWarnings - we don't need to worry about them in this case
#(This is run using Python 3.9)

%load_ext autoreload
%autoreload 2

plt.rcParams['font.size']=12
plt.rcParams['font.family']='serif'
plt.rcParams['axes.spines.top'] = False
plt.rcParams['axes.spines.right'] = False  
plt.rcParams['axes.spines.bottom'] = True     
plt.rcParams["axes.grid"] =True
plt.rcParams['grid.linestyle'] = '-.' 
plt.rcParams['grid.linewidth'] = 0.4

The autoreload extension is already loaded. To reload it, use:
  %reload_ext autoreload


## Reading data

In [22]:
S_base_3ph = 100
gen_data = fnc.read_data('gen_data')
demand = fnc.read_data('system_demand')['System Demand']
load_distribution = fnc.read_data('load_distribution')
gen_costs = fnc.read_data('gen_costs')
line_data = fnc.read_data('line_data')
branch_matrix = fnc.read_data('branch_matrix')
wind_data = fnc.read_data('wind_data')

GC = len(gen_data.index)
GW = len(wind_data.columns)
G = GC + GW
D = len(load_distribution.index)
N = 24 # number of buses

WF_NODES = [2, 4, 6, 15, 20, 22] #zero-indexed

Set up demand array for the hour

In [19]:
t = 0 #hour
load = np.zeros(N)

#Saving the load for each bus in a numpy array accounting for the system load destribution
for n in load_distribution['Node'].unique():
    load[n-1] = load_distribution.loc[load_distribution['Node'] == n, r'% of system load'] / 100 * demand[t] #per-unitized load - remember that the data is not 0-indexed but the arrays are

load

array([ 67.48173 ,  60.37839 , 111.877605,  46.17171 ,  44.395875,
        85.24008 ,  78.13674 , 106.5501  , 108.325935, 120.75678 ,
         0.      ,   0.      , 165.152655, 120.75678 , 197.117685,
        62.154225,   0.      , 207.772695, 113.65344 ,  79.912575,
         0.      ,   0.      ,   0.      ,   0.      ])

Setup bid prices for loads

In [ ]:
#...

## Setup Gurobi model

In [ ]:
direction = gb.GRB.MAXIMIZE #Min / Max

# Create a Gurobi model       
m = gb.Model()

# Add variables
p_G = m.addVars(G, lb=0, ub=gb.GRB.INFINITY, name="P_G") #Note: wind farms can be curtailed AND P_min from the system should be disregarded to avoid having a mixed integer program
p_D = m.addVars(D, lb=0, ub=gb.GRB.INFINITY, name="P_D") #Note: demands are elastic

# Set objective function (social welfare)
obj = gb.quicksum(bid_prices[d] * p_D[d] for d in range(D)) - gb.quicksum(gen_costs['C (DKK/MWh)'][k] * p_G[k] for k in range(G))
m.setObjective(obj, direction)

#============= Balance equation =============
m.addConstr(gb.quicksum(p_G[g] for g in range(G)) - gb.quicksum(p_D[d] for d in range(D)) == 0)

#============= Generator limits =============
m.addConstrs(p_G[g] <= (gen_data['P max MW'].iloc[g]) for g in range(GC)) #conventinal generator upper limits

#Maximum wind power is the per-unitized output of the non-curtailed wind farm in the hour t
m.addConstrs(p_G[GC + g] <= wind_data.iloc[t, g] for g in range(GW)) #wind farm generator upper limits

#============= Demand limits =============
m.addConstrs(p_D[d] <= load[d] for d in range(D)) #demand limits

#============= Display and run model =============
m.update()
m.display()
m.optimize()

## Analyze results

In [ ]:
#Print solutions
if m.status == gb.GRB.OPTIMAL:
    results = {}
    generator_outputs = pd.DataFrame(data=np.zeros(G), columns=['p_G'])
    generator_outputs['Node'] = 0 #initialize
    generator_outputs['Dispatched Percentage'] = 0

    load_results = pd.DataFrame(data=np.zeros(D), columns=['p_D'])
    generator_outputs['Node'] = load_distribution['Node'].values


    constraints = m.getConstrs()
    # The constraint dual value of the current solution (also known as the shadow price)... https://www.gurobi.com/documentation/9.5/refman/pi.html
    dual_values = [constraints[k].Pi for k in range(len(constraints))] 
    print('-----------------------------------------------')
    print("Optimal objective value: %.2f DKK" % m.objVal)

    for i in range(G):
        print(p_G[i].VarName + ": %.2f pu" % p_G[i].x)
        generator_outputs.loc[generator_outputs.index == i, 'p_G'] = p_G[i].x

        if i < GC:
            generator_outputs.loc[generator_outputs.index == i, 'Node'] = gen_data.loc[gen_data.index == i, 'Node'].values #save node
            gen_limit = gen_data.loc[gen_data.index == i, 'P max MW']
            generator_outputs.loc[generator_outputs.index == i, 'Dispatched Percentage'] = (p_G[i].x / gen_limit) * 100
        else:
            generator_outputs.loc[generator_outputs.index == i, 'Node'] =
            wind_limit = wind_data.iloc[t, i - GC]
            generator_outputs.loc[generator_outputs.index == i, 'Dispatched Percentage'] = (p_G[i].x / wind_limit) * 100

    for i in range(D):
        print(p_D[i].VarName + ": %.2f pu" % p_D[i].x)


    for k in range(len(constraints)):
        print('Dual value {0}: '.format(k+1), dual_values[k])

    sum_gen = sum(p_G[n].x for n in range(G))
    sum_load = sum(p_D[n].x for n in range(D))
    print("\nTotal load: %.1f MWh" % sum_load)
    print("Total generation: %.1f MWh" % sum_gen)

    results['gen'] = generator_outputs.copy(deep=True)
    results['demand'] = generator_outputs.copy(deep=True)

else:
    print("Optimization was not successful.")     
  
        
# Note that the dual values will reflect a per-unit basis. So if the LMPs are 3845, then it is for a step in demand of 1 pu = 100 MWh. So the value should be divided by 100 MW.

## Visualization

In [ ]:
fig, ax = plt.subplots(1,2,sharey=False,figsize=( 15 , 5 ), dpi=400) # Create the figure

(results.get('wind')['p_W']).plot(kind='bar', ax=ax[0], color='teal', width=0.2, label='Dispatched', edgecolor='darkslategrey')
(results.get('wind')['Expected Production']).plot(kind='bar', ax=ax[0], width=0.2, edgecolor='black', linestyle='--', fill=False, linewidth=1.5, label='Expected Production')
ax[0].set_ylabel('Power [MW]')
ax[0].set_xlabel('Wind Farm')
ax[0].set_xticks(ticks = np.arange(0, n_wf), labels = np.arange(1, n_wf + 1), fontsize=12, rotation = 0)
ax[0].legend(loc='upper left', ncol=2, fontsize=12)
ax[0].set_ylim([0,110])

(results.get('gen')['p_G']).plot(kind='bar', ax=ax[1], color='darkgreen', label='Dispatched', edgecolor='darkslategrey')
#((results.get('gen')['p_G']) / ((results.get('gen')['Loading Percentage']) / 100)).plot(kind='bar', ax=ax[1], edgecolor='black', linestyle='-', fill=False, linewidth=1.2, label='Limit')
gen_data['P max MW'].plot(kind='bar', ax=ax[1], edgecolor='black', linestyle='-', fill=False, linewidth=1.2, label='Limit')
ax[1].yaxis.tick_right()
ax[1].yaxis.set_label_position("right")
ax[1].set_ylabel('Power [MW]')
ax[1].set_xlabel('Generator')
ax[1].set_xticks(ticks = np.arange(0, n_gen), labels = np.arange(1, n_gen + 1), fontsize=12, rotation = 0)
ax[1].legend(loc='upper right', ncol=2, fontsize=12)

for k in range(2):
    ax[k].spines[['right', 'top']].set_visible(True)
    ax[k].set_axisbelow(True)

fig.tight_layout() # reduces white space around figures
#plt.savefig('Figures/task_1_results_gen_dispatch.png',bbox_inches='tight')
plt.show()